In [1]:
import pandas as pd
import numpy as np

from itertools import groupby
from sklearn.preprocessing import MultiLabelBinarizer

import json
import glob
import ast

from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [2]:
def openfile(data):
    if data == "writers":
        with open("writing.json") as f:
            writers = f.read()
        
        return json.loads(writers)
    elif data == "directors":
        with open("directing.json") as f:
            directors = f.read()
        
        return json.loads(directors)    
    else:
        return NotImplemented    

In [3]:
def json_to_one_hot(kind = "writers"):

    if kind in ["writers", "directors"]:
        json_file = openfile(kind)
    else:
        return NotImplemented
  
    if kind == "writers":
         # Group writers/directors by movie
        groups = groupby([(i["movie"], i["writer"]) for i in json_file], key = lambda x : x[0])

        # Turn groupby object into a json-like dict
        grouped = {writer: [i[1] for i in movies] for writer, movies in groups}

        # Convert to Series
        df = pd.DataFrame.from_records(list(grouped.items())).set_index(0).squeeze()
    else:
        df = pd.DataFrame(json_file).groupby("movie")["director"].apply(lambda x: x.values)
    
    # Create one-hot encoded DataFrame
    mlb = MultiLabelBinarizer()
    res = pd.DataFrame(mlb.fit_transform(df),
                       columns=mlb.classes_,
                       index=df.index)
    
    return res.drop("\\N", axis = 1)

In [4]:
all_files = glob.glob("train*.csv")

print(f"Found files: {', '.join(all_files)}")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True).drop("Unnamed: 0", axis = 1)

Found files: train-1.csv, train-2.csv, train-3.csv, train-4.csv, train-5.csv, train-6.csv, train-7.csv, train-8.csv


In [5]:
df = df.replace("\\N", np.nan)

In [6]:
df.head()

,tconst,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,numVotes,label
0,tt0010600,The Doll,Die Puppe,1919,NaN,66,1898.0,True
1,tt0011841,Way Down East,Way Down East,1920,NaN,145,5376.0,True
2,tt0012494,Déstiny,Der müde Tod,1921,NaN,97,5842.0,True
3,tt0015163,The Navigator,The Navigator,1924,NaN,59,9652.0,True
4,tt0016220,The Phantom of the Opera,The Phantom of the Opera,1925,NaN,93,17887.0,True


In [7]:
df["label"].value_counts()

True     3990
False    3969
Name: label, dtype: int64

In [8]:
df.isna().mean()

tconst            0.000000
primaryTitle      0.000000
originalTitle     0.501068
startYear         0.098756
endYear           0.901244
runtimeMinutes    0.001633
numVotes          0.099259
label             0.000000
dtype: float64

In [9]:
oscars = pd.read_csv("additional_data/oscars.csv")

In [10]:
oscars.head()

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False


In [11]:
# Cleaning titles and merging with oscar noms
df["primaryTitle"] = df["primaryTitle"].str.lower()\
                                       .str.normalize('NFKD')\
                                       .str.encode('ascii', errors='ignore')\
                                       .str.decode('utf-8')\
                                       .str.replace(" ", "_", regex=True)\
                                       .str.replace("\W", "", regex=True)

oscars["film"] = oscars["film"].str.lower()\
                               .str.normalize('NFKD')\
                               .str.encode('ascii', errors='ignore')\
                               .str.decode('utf-8')\
                               .str.replace(" ", "_", regex=True)\
                               .str.replace("\W", "", regex=True)

In [12]:
# Counting oscar nominations and wins per movie
oscar_noms = pd.merge(df, oscars, left_on = "primaryTitle", right_on = "film").groupby("tconst")["winner"].count()
oscar_wins = pd.merge(df, oscars, left_on = "primaryTitle", right_on = "film").groupby("tconst")["winner"].sum()

In [13]:
df = df.rename({"tconst" : "id"}, axis = 1).set_index("id")

In [14]:
df["oscar_noms"] = oscar_noms
df["oscar_wins"] = oscar_wins

In [15]:
# Find writers and directors per movie and combine the two
written_and_directed = (json_to_one_hot("writers") + json_to_one_hot("directors")).fillna(0).astype(int).loc[df.index]

In [16]:
# Add writer/director data to df (the transposes are done to speed things up)
df = pd.concat([df.T, written_and_directed.T]).T

In [17]:
df.head()

,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,numVotes,label,oscar_noms,oscar_wins,nm0000005,...,nm9925241,nm9933959,nm9942830,nm9946633,nm9955258,nm9958352,nm9958353,nm9980769,nm9985316,nm9985837
id,,,,,,,,,,,,,,,,,,,,,
tt0010600,the_doll,Die Puppe,1919,NaN,66,1898.0,True,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,0
tt0011841,way_down_east,Way Down East,1920,NaN,145,5376.0,True,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,0
tt0012494,destiny,Der müde Tod,1921,NaN,97,5842.0,True,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,0
tt0015163,the_navigator,The Navigator,1924,NaN,59,9652.0,True,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,0
tt0016220,the_phantom_of_the_opera,The Phantom of the Opera,1925,NaN,93,17887.0,True,3.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0


## Add TMDB data

In [18]:
df_2 = pd.read_csv("additional_data/TMDB.csv")

In [19]:
df_2 = df_2[["budget", "genres", "imdb_id", 
             "original_language", "overview", 
             "popularity", "production_companies", 
             "tagline", "Keywords", "revenue"]]

In [20]:
def dict_to_list(dictionary):
    try:
        d = ast.literal_eval(dictionary)
    except ValueError:
        return []
    
    return [i["name"] for i in d]

In [21]:
df_2["genres"] = df_2["genres"].apply(lambda x: dict_to_list(x))

In [22]:
df_2["Keywords"] = df_2["Keywords"].apply(lambda x: dict_to_list(x))

In [23]:
df_2["production_companies"] = df_2["production_companies"].apply(lambda x: dict_to_list(x))

In [24]:
df_full = pd.merge(df.reset_index(), df_2, how = "left", left_on = "id", right_on = "imdb_id").set_index("id")

In [25]:
del df

In [48]:
def embed_overviews(df_full, vector_size, epochs):
    df = df_full.copy()
    
    df["overview"] = df["overview"].str.lower().str.split()
    
    texts = df[~df["overview"].isna()]["overview"]
        
    documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(texts)]
    model = Doc2Vec(documents, vector_size = vector_size, epochs = epochs, min_count = 1)
        
    embeddings = df["overview"].apply(lambda x: 
                                      model.infer_vector(x)
                                      if type(x) == list
                                      else [np.nan] * vector_size)
                 
    return embeddings
    
vector_size = 10
epochs = 10
embeddings = pd.DataFrame(embed_overviews(df_full, vector_size, epochs).to_list(), 
                          columns = [f"embedding_{i}" for i in range(vector_size)]).set_index(df_full.index)

In [52]:
df_full = pd.merge(df_full, embeddings, left_index=True, right_index=True)

In [53]:
df_full.head(20)

,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,numVotes,label,oscar_noms,oscar_wins,nm0000005,...,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,embedding_9
id,,,,,,,,,,,,,,,,,,,,,
tt0010600,the_doll,Die Puppe,1919,NaN,66,1898.0,True,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tt0011841,way_down_east,Way Down East,1920,NaN,145,5376.0,True,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tt0012494,destiny,Der müde Tod,1921,NaN,97,5842.0,True,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tt0015163,the_navigator,The Navigator,1924,NaN,59,9652.0,True,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tt0016220,the_phantom_of_the_opera,The Phantom of the Opera,1925,NaN,93,17887.0,True,3.0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tt0016630,battling_butler,Battling Butler,1926,NaN,77,3285.0,True,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tt0021015,juno_and_the_paycock,NaN,1929,NaN,85,2275.0,False,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tt0023973,the_eagle_and_the_hawk,NaN,1933,NaN,73,NaN,True,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tt0023986,employees_entrance,NaN,1933,NaN,75,NaN,True,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Inspection of numVotes

In [29]:
df[df['numVotes'].isna()]

NameError: name 'df' is not defined

In [ ]:
type(df.loc['tt0029146']['numVotes'])

In [ ]:
df[['numVotes', 'label']]

In [ ]:
df['numVotes'] = df['numVotes'].fillna(0)
df["label"] = df["label"].astype(int)

In [ ]:
abs(df["label"].corr(df["numVotes"]))

In [ ]:
df.corr()

In [ ]:
display(df.sort_values(by='numVotes', ascending=False).head(100))
df.sort_values(by='numVotes', ascending=False).head(1000).corr()